<h1 style="text-align: center;">Causal Inference with DoubleML: Job Training Impact Analysis</h1>

<br>

## Initial Setup

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from doubleml import DoubleMLPLR, DoubleMLIRM
from doubleml import DoubleMLData
from scipy import stats

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

<br>

## Load and Prepare Dataset

---

In [ ]:
# Load the LaLonde dataset
url = "https://raw.githubusercontent.com/mdcattaneo/replication-CCJM_2022_RESTAT/main/lalonde.csv"
df = pd.read_csv(url)

# Prepare the data
y_col = 're78'  # 1978 earnings (outcome)
d_col = 'treat'  # Treatment indicator
x_cols = ['age', 'educ', 'black', 'hisp', 'married', 'nodegree', 're74', 're75']

# Create DoubleMLData object
dml_data = DoubleMLData(df, y_col, d_col, x_cols)

# Standardize continuous variables
scaler = StandardScaler()
df[['age', 'educ', 're74', 're75']] = scaler.fit_transform(df[['age', 'educ', 're74', 're75']])


<br>

## Exploratory Data Analysis (EDA)

---

In [ ]:
# Basic statistics
print(df.describe())

# Treatment group sizes
print(df['treat'].value_counts(normalize=True))

# Correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(df[x_cols + [y_col, d_col]].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

# Distribution of outcome variable by treatment group
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x=y_col, hue=d_col, element='step', stat='density', common_norm=False)
plt.title('Distribution of 1978 Earnings by Treatment Group')
plt.show()

<br>

## DoubleML Estimation

---

In [ ]:
# Specify the machine learning models
ml_g = RandomForestRegressor(n_estimators=100, max_depth=5)
ml_m = RandomForestClassifier(n_estimators=100, max_depth=5)

# Create and fit DoubleMLPLR model
dml_plr = DoubleMLPLR(dml_data, ml_g, ml_m)
dml_plr.fit()

print("Partial Linear Regression Results:")
print(dml_plr.summary)

# Create and fit DoubleMLIRM model (Interactive Regression Model)
dml_irm = DoubleMLIRM(dml_data, ml_g, ml_m)
dml_irm.fit()

print("\nInteractive Regression Model Results:")
print(dml_irm.summary)

<br>

## Comparison with Other Methods

---

In [ ]:
# Naive estimate (simple difference in means)
naive_ate = df[df[d_col] == 1][y_col].mean() - df[df[d_col] == 0][y_col].mean()

print(f"Naive ATE estimate: ${naive_ate:.2f}")

# Simple OLS regression
import statsmodels.api as sm

X = sm.add_constant(df[x_cols + [d_col]])
ols_model = sm.OLS(df[y_col], X).fit()
print("\nOLS Regression Results:")
print(ols_model.summary())

<br>

## Additional Analyses

---

In [ ]:
# Heterogeneous treatment effects
def het_treat_effect(df, feature):
    grouped = df.groupby(feature)
    effects = grouped.apply(lambda x: x[x[d_col] == 1][y_col].mean() - x[x[d_col] == 0][y_col].mean())
    return effects

# Calculate heterogeneous effects for education
het_effects_educ = het_treat_effect(df, 'educ')

plt.figure(figsize=(10, 6))
het_effects_educ.plot(kind='bar')
plt.title('Heterogeneous Treatment Effects by Education Level')
plt.xlabel('Education (years)')
plt.ylabel('Estimated Treatment Effect')
plt.show()

# Sensitivity analysis
def sensitivity_analysis(model, n_simulations=1000):
    coef_samples = np.random.normal(model.coef[0], model.se[0], n_simulations)
    return pd.Series(coef_samples).describe()

print("\nSensitivity Analysis for DoubleMLPLR:")
print(sensitivity_analysis(dml_plr))

# Propensity score analysis
propensity_model = RandomForestClassifier(n_estimators=100, max_depth=5)
propensity_model.fit(df[x_cols], df[d_col])
df['propensity_score'] = propensity_model.predict_proba(df[x_cols])[:, 1]

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='propensity_score', hue=d_col, element='step', stat='density', common_norm=False)
plt.title('Distribution of Propensity Scores by Treatment Group')
plt.show()

<br>

## Results Interpretation

---

In [ ]:
def interpret_results(model):
    ate = model.coef[0]
    se = model.se[0]
    ci_lower, ci_upper = model.confint()[0]
    
    print(f"Estimated Average Treatment Effect: ${ate:.2f}")
    print(f"95% Confidence Interval: (${ci_lower:.2f}, ${ci_upper:.2f})")
    print(f"P-value: {model.pval[0]:.4f}")
    
    if model.pval[0] < 0.05:
        print("The treatment effect is statistically significant at the 5% level.")
    else:
        print("The treatment effect is not statistically significant at the 5% level.")
    
    print(f"This suggests that, on average, the job training program ")
    print(f"{'increased' if ate > 0 else 'decreased'} 1978 earnings by ${abs(ate):.2f}.")

print("\nInterpretation of DoubleMLPLR results:")
interpret_results(dml_plr)